# Normative 2b -- Compute the baseline decodability of Motor responses (LINDEX v. LMID and RINDEX v. RMID)
## Using normative transformations, region-to-region
## Use 100 PCs to make transforms more tractable

## Use SVM classifications to decode hand-specific responses
## Using Ciric-style postprocessing

## Takuya Ito
#### 12/12/2018

In [13]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns
import h5py
os.sys.path.append('glmScripts/')
import taskGLMPipeline as tgp
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.decomposition import PCA
import time



sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.0 Define functions for loading data

In [17]:
def loadMotorResponses(subj,hand='Right'):
    
    hands = {'Left':[0,1],'Right':[2,3]}

    x = tgp.loadTaskTiming(subj,'ALL')
    stimIndex = np.asarray(x['stimIndex'])
    ind = np.where(stimIndex=='motorResponse')[0]
    
    datadir = basedir + 'data/postProcessing/hcpPostProcCiric/'
    h5f = h5py.File(datadir + subj + '_glmOutput_data.h5','r')
    data = h5f['taskRegression/ALL_24pXaCompCorXVolterra_taskReg_betas_canonical'][:].copy()
    data = data[:,ind].copy()
    h5f.close()
    
    # Isolate hand responses
    hand_ind = hands[hand]
    data = data[:,hand_ind]
    
    return data

def loadRSFCMapping(subj,roi):
    fcdir = '/projects3/SRActFlow/data/results/ridgeFC/'
    filename = fcdir + 'TargetParcel' + str(roi) + '_RidgeFC.h5'
    h5f = h5py.File(filename,'r')
    fcmapping = h5f[subj]['sourceToTargetMapping'][:].copy()
    h5f.close()
    return fcmapping

def ridgeWrapper((stim,resp,alpha)):
#     wt = ridge.ridge.ridge(stim,resp,alpha)
    clf = Ridge(alpha=alpha)
    clf.fit(stim,resp)
    wt = clf.coef_

    return wt

def pcaFC(stim,resp,n_components=500,nproc=10):
#     print '\tRunning PCA'
    os.environ['OMP_NUM_THREADS'] = str(nproc)
    if n_components<stim.shape[1]:
        pca = PCA(n_components)
        reduced_mat = pca.fit_transform(stim) # Time X Features
    else:
        reduced_mat = stim
    
    inputs = []
    for vert in range(resp.shape[1]):
        inputs.append((resp[:,vert],reduced_mat,True))

#     print '\tRunning regression'
    os.environ['OMP_NUM_THREADS'] = str(1)
    pool = mp.Pool(processes=nproc)
    results = pool.map_async(_regression2,inputs).get()
    pool.close()
    pool.join()
    
    
    wt = np.zeros((stim.shape[1],resp.shape[1]))
    vert = 0
    for result in results:
        betas, resid = result
        if n_components<stim.shape[1]:
            betas = pca.inverse_transform(betas[1:])
        else:
            betas = betas[1:]
        wt[:,vert] = betas
        vert += 1

    return wt

## Load masks
def loadMask(roi,dilated=True):
    maskdir = basedir + 'data/results/surfaceMasks/'
    if dilated:
        maskfile = maskdir + 'GlasserParcel' + str(roi) + '_dilated_10mm.dscalar.nii'
    else:
        maskfile = maskdir + 'GlasserParcel' + str(roi) + '.dscalar.nii'
    maskdata = np.squeeze(nib.load(maskfile).get_data())
    return maskdata

def loadRegularizationTerms(roi):
    ridgefcdir = '/projects3/SRActFlow/data/results/ridgeFC/'
    filename = ridgefcdir + 'TargetParcel' + str(roi) + '_RidgeFC.h5'
    h5f = h5py.File(filename,'r')
    alphas = h5f['alphasPerVertex'][:].copy()
    return alphas


def _regression2((data,regressors,constant)):
    """ 
    Hand coded OLS regression using closed form equation: betas = (X'X)^(-1) X'y
    """
    # Add 'constant' regressor
    if constant:
        regressors = sm.add_constant(regressors)
    X = regressors.copy()
    try:
#        #C_ss_inv = np.linalg.inv(np.dot(X.T,X))
        C_ss_inv = np.linalg.pinv(np.dot(X.T,X))
    except np.linalg.LinAlgError as err:
        C_ss_inv = np.linalg.pinv(np.cov(X.T))
    betas = np.dot(C_ss_inv,np.dot(X.T,data.T))
    resid = data - (betas[0] + np.dot(X[:,1:],betas[1:])).T
    return betas, resid

        

## 0.1 Load data

In [4]:
# gsr = True
nResponses = 2
data_task = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadMotorResponses(subj, hand='Right')
    scount += 1

## 0.2 Generate normative transformation

In [20]:
os.environ['OMP_NUM_THREADS'] = str(20)

roi_rh = 9
roi_lh = 189
actflow_data = np.zeros((len(glasser2),nResponses,len(subjNums),nParcels))

dilateLH = loadMask(roi_lh,dilated=True)
dilateRH = loadMask(roi_rh,dilated=True)
combinedDilated = dilateLH + dilateRH
source_space = np.where(combinedDilated==0)[0]

target_ind = np.where(glasser2==roi_rh)[0]
normativedir = '/projects3/SRActFlow/data/results/normativeFC/'
h5f = h5py.File(normativedir + 'TargetParcel' + str(roi_rh) + '_NormativePCA_FC.h5','a')


scount = 0
for subj in subjNums:
    print 'Subject', subj, '(', scount+1, '/', len(subjNums), ')'
    training_subjs = np.delete(np.arange(len(subjNums)),scount)
    before = time.time()
    
    bad_rois = []
    for roi in range(nParcels):
#         print '\tROI number', roi+1
        source_ind = np.where(glasser2==roi+1)[0]
        source_ind = np.intersect1d(source_ind,source_space) # Make sure no vertices are within 10mm of the target
        if source_ind.shape[0]==0:
            bad_rois.append(roi)
            continue

        targetTrain = data_task[:,:,training_subjs].T.reshape(len(training_subjs)*2,len(glasser2))[:,target_ind]
        sourceTrain = data_task[:,:,training_subjs].T.reshape(len(training_subjs)*2,len(glasser2))[:,source_ind]
#         normativeMapping = pcaFC(sourceTrain,targetTrain,n_components=50,nproc=20)
#         try:
#             h5f.create_dataset(subj + '/' + 'Source' + str(roi+1), data=normativeMapping)
#         except:
# #             del h5f[subj + '/' + 'Source' + str(roi+1)]
#             h5f.create_dataset(subj + '/' + 'Source' + str(roi+1), data=normativeMapping)
      
        # Right Finger 1
        actflow_data[target_ind,0,scount,roi] = np.dot(stats.zscore(data_task[source_ind,0,scount],axis=0),normativeMapping[:,:])
        # Right Finger 2
        actflow_data[target_ind,1,scount,roi] = np.dot(stats.zscore(data_task[source_ind,1,scount],axis=0),normativeMapping[:,:])
        
    after = time.time()
    print '...Elapsed time:', after-before
    
    scount += 1
    
h5f.close()

Subject 013 ( 1 / 96 )
...Elapsed time: 713.768088102
Subject 014 ( 2 / 96 )
...Elapsed time: 483.543848991
Subject 016 ( 3 / 96 )
...Elapsed time: 542.476314068
Subject 017 ( 4 / 96 )
...Elapsed time: 620.584634066
Subject 018 ( 5 / 96 )
...Elapsed time: 669.172348022
Subject 021 ( 6 / 96 )
...Elapsed time: 520.715664864
Subject 023 ( 7 / 96 )
...Elapsed time: 435.62855792
Subject 024 ( 8 / 96 )
...Elapsed time: 382.276139021
Subject 026 ( 9 / 96 )
...Elapsed time: 384.091159821
Subject 027 ( 10 / 96 )
...Elapsed time: 376.547902107
Subject 028 ( 11 / 96 )
...Elapsed time: 370.555563927
Subject 030 ( 12 / 96 )
...Elapsed time: 365.759562016
Subject 031 ( 13 / 96 )
...Elapsed time: 371.390974998
Subject 032 ( 14 / 96 )
...Elapsed time: 377.802261114
Subject 033 ( 15 / 96 )
...Elapsed time: 373.220960855
Subject 034 ( 16 / 96 )
...Elapsed time: 372.640937805
Subject 035 ( 17 / 96 )
...Elapsed time: 374.193254948
Subject 037 ( 18 / 96 )
...Elapsed time: 373.651030064
Subject 038 ( 19 / 9

# 1.0 Define functions for motor response decodings

In [21]:
from scipy.spatial.distance import cdist

def motorResponseDecodings(data, actflow_data, rois=[9], ncvs=1, nproc=5):
    """
    Run an across-subject classification
    Decode responses on each hand separately from CPRO data
    """

    nSubjs = data.shape[2]
    stats = np.zeros((len(rois),nSubjs*2))
    
    nfing = data.shape[1]

    nsamples = nSubjs * nfing

    # Label array for supervised learning
    labels = np.tile(range(nfing),nSubjs)
    subjarray = np.repeat(range(nSubjs),nfing)

    # Run SVM classifications on network-level activation patterns across subjects
    roicount = 0
    for roi in rois:
        roi_ind = np.where(glasser2==roi)[0]
        nfeatures = len(roi_ind)
        roi_ind.shape = (len(roi_ind),1)       

        svm_mat = np.zeros((nsamples,roi_ind.shape[0]))
        actflow_svm_mat = np.zeros((nsamples,roi_ind.shape[0]))
        samplecount = 0
        scount = 0
        for subj in range(len(subjNums)):
            roidata = np.squeeze(data[roi_ind,:,scount])
            actflow_roidata = np.squeeze(actflow_data[roi_ind,:,scount])
            svm_mat[samplecount:(samplecount+nfing),:] = roidata.T
            actflow_svm_mat[samplecount:(samplecount+nfing),:] = actflow_roidata.T

            scount += 1
            samplecount += nfing

#             # Spatially demean matrix across features
#             samplemean = np.mean(svm_mat,axis=1)
#             samplemean.shape = (len(samplemean),1)
#             svm_mat = svm_mat - samplemean

        scores = randomSplitLOOBaselineCV(ncvs, svm_mat, actflow_svm_mat, labels, subjarray, nproc=nproc)
        stats[roicount,:] = scores
        roicount += 1
        
    return stats

def randomSplitLOOBaselineCV(ncvs, svm_mat, actflow_svm_mat, labels, subjarray, nproc=5):
    """
    Runs cross validation for an across-subject SVM analysis
    """
    
    ntasks = len(np.unique(labels))
    nsamples = svm_mat.shape[0]
    nsubjs = nsamples/ntasks

    subjects = np.unique(subjarray)
    indices = np.arange(nsamples)
    
    numsubjs_perfold = 4
    if nsubjs%numsubjs_perfold!=0: 
        raise Exception("Error: Folds don't match number of subjects")
        
    nfolds = nsubjs/numsubjs_perfold
    subj_array_folds = subjarray.copy()
    
    inputs = [] 
    
    for fold in range(nfolds):
        test_subjs = np.random.choice(subj_array_folds,numsubjs_perfold,replace=False)
        train_subjs_all = np.delete(subjects,test_subjs)
        for cv in range(ncvs):
            # Randomly sample half of train set subjects for each cv (CV bootstrapping)
            train_subjs = np.random.choice(train_subjs_all,
                                         int(np.floor(len(train_subjs_all)*(2.0))),
                                         replace=True)

            train_ind = []
            for subj in train_subjs:
                train_ind.extend(np.where(subjarray==subj)[0])

            test_ind = []
            for subj in test_subjs:
                test_ind.extend(np.where(subjarray==subj)[0])
            
            train_ind = np.asarray(train_ind)
            test_ind = np.asarray(test_ind)

            trainset = actflow_svm_mat[train_ind,:]
            testset = svm_mat[test_ind,:]

            # Normalize trainset and testset
            trainmean = np.mean(actflow_svm_mat[train_ind,:],axis=0)
            trainmean.shape = (1,len(trainmean))
            trainstd = np.std(actflow_svm_mat[train_ind,:],axis=0)
            trainstd.shape = (1,len(trainstd))
            
            # Normalize trainset and testset
            testmean = np.mean(svm_mat[train_ind,:],axis=0)
            testmean.shape = (1,len(testmean))
            teststd = np.std(svm_mat[train_ind,:],axis=0)
            teststd.shape = (1,len(teststd))

            trainset = np.divide((trainset - trainmean),trainstd)
            testset = np.divide((testset - testmean),teststd)

            inputs.append((trainset,testset,labels[train_ind],labels[test_ind]))
        
        subj_array_folds = np.delete(subj_array_folds,test_subjs)
        
    pool = mp.Pool(processes=nproc)
    scores = pool.map_async(_decoding,inputs).get()
    pool.close()
    pool.join()
    
#     subj_acc = np.zeros((len(subjects),))
#     scount = 0
#     i = 0
#     for subj in subjects:
#         subjmean = []
#         for cv in range(ncvs):
#             subjmean.append(scores[i])
#             i += 1
        
#         subj_acc[scount] = np.mean(subjmean)
        
#         scount += 1

#     return subj_acc

    acc = []
    for score in scores:
        acc.extend(score)
    return acc

def _decoding((trainset,testset,trainlabels,testlabels)):

#     clf = sklearn.linear_model.LogisticRegression()
    clf = svm.SVC(C=1.0, kernel='linear')

    clf.fit(trainset,trainlabels)
    predictions = clf.predict(testset)
    acc = predictions==testlabels
    
#     unique_cond = np.unique(trainlabels)
#     rdm = np.zeros((len(unique_cond),len(unique_cond)))
#     acc = []
#     for cond1 in unique_cond:
#         mismatches = []
#         prototype_ind = np.where(trainlabels==cond1)[0]
#         prototype = np.mean(trainset[prototype_ind,:],axis=0)
#         for cond2 in unique_cond:
#             test_ind = np.where(testlabels==cond2)[0]
#             test = np.mean(testset[test_ind,:],axis=0)
#             if cond1 == cond2: 
#                 correct = stats.pearsonr(prototype,test)[0]
#             else:
#                 mismatches.append(stats.pearsonr(prototype,test)[0])
        
#         if correct > np.max(mismatches): 
#             acc.append(1.0)
#         else:
#             acc.append(0.0)
    
    return acc

## 1.1 Run across subject decoding on right-hand motor responses

In [23]:
nproc = 20
ncvs = 1

rois = np.where(networkdef==networkmappings['smn'])[0] + 1
rois = [9] # Left S1


distances_baseline_rh = []
for roi in range(nParcels):
    print 'Decoding roi', roi
    if roi in bad_rois: 
        distances_baseline_rh.append(np.zeros((1,len(subjNums)*2.0)))
        continue

    distances_baseline_rh.append(motorResponseDecodings(stats.zscore(data_task,axis=0), 
                                                     stats.zscore(actflow_data[:,:,:,roi],axis=0),
                                                     rois=rois, ncvs=ncvs, nproc=nproc))
    
distances_baseline_rh = np.squeeze(np.asarray(distances_baseline_rh))

Decoding roi 0
Decoding roi 1
Decoding roi 2
Decoding roi 3
Decoding roi 4
Decoding roi 5
Decoding roi 6
Decoding roi 7
Decoding roi 8
Decoding roi 9


/home/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if sys.path[0] == '':


Decoding roi 10
Decoding roi 11
Decoding roi 12
Decoding roi 13
Decoding roi 14
Decoding roi 15
Decoding roi 16
Decoding roi 17
Decoding roi 18
Decoding roi 19
Decoding roi 20
Decoding roi 21
Decoding roi 22
Decoding roi 23
Decoding roi 24
Decoding roi 25
Decoding roi 26
Decoding roi 27
Decoding roi 28
Decoding roi 29
Decoding roi 30
Decoding roi 31
Decoding roi 32
Decoding roi 33
Decoding roi 34
Decoding roi 35
Decoding roi 36
Decoding roi 37
Decoding roi 38
Decoding roi 39
Decoding roi 40
Decoding roi 41
Decoding roi 42
Decoding roi 43
Decoding roi 44
Decoding roi 45
Decoding roi 46
Decoding roi 47
Decoding roi 48
Decoding roi 49
Decoding roi 50
Decoding roi 51
Decoding roi 52
Decoding roi 53
Decoding roi 54
Decoding roi 55
Decoding roi 56
Decoding roi 57
Decoding roi 58
Decoding roi 59
Decoding roi 60
Decoding roi 61
Decoding roi 62
Decoding roi 63
Decoding roi 64
Decoding roi 65
Decoding roi 66
Decoding roi 67
Decoding roi 68
Decoding roi 69
Decoding roi 70
Decoding roi 71
Decoding

## 1.2 Compute statistics

In [27]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],3))
for roicount in range(distances_baseline_rh.shape[0]):
    ntrials = len(subjNums)*2
    p = stats.binom_test(np.mean(distances_baseline_rh[roicount,:])*ntrials,n=ntrials,p=0.5)
    if np.mean(distances_baseline_rh[roicount,:])>0.5:
        p = p/2.0
    else:
        p = 1.0-p/2.0

    statistics_rh[roicount,0] = np.mean(distances_baseline_rh[roicount,:])
    statistics_rh[roicount,1] = p

rois_testing = []
rois_testing.extend(np.where(networkdef==networkmappings['fpn'])[0])
rois_testing.extend(np.where(networkdef==networkmappings['con'])[0])
rois_testing.extend(np.where(networkdef==networkmappings['dan'])[0])
# rois_notSMN = np.where(networkdef!=networkmappings['smn'])[0]
h0, qs = mc.fdrcorrection0(statistics_rh[rois_testing,1])
statistics_rh[:,1] = 1.0
i = 0
for roi in rois_testing:
    statistics_rh[roi,1] = qs[i]
    statistics_rh[roi,2] = h0[i]*statistics_rh[roi,0]
    i += 1
        
nSignificant = np.sum(statistics_rh[:,1] < 0.05)
print 'Number of significant transfers:', nSignificant

if nSignificant>0:
    sig_ind = np.where(statistics_rh[:,1]<0.05)[0]
    for ind in sig_ind:
        print '\tSignificant parcel:', ind+1
        print '\tAccuracy:', statistics_rh[ind,0]
        print '\tNetwork:', networkdef[ind]
print 'Network mappings:', networkmappings

Number of significant transfers: 15
	Significant parcel: 54
	Accuracy: 0.572916666667
	Network: 3.0
	Significant parcel: 79
	Accuracy: 0.5625
	Network: 6.0
	Significant parcel: 103
	Accuracy: 0.578125
	Network: 8.0
	Significant parcel: 142
	Accuracy: 0.572916666667
	Network: 1.0
	Significant parcel: 173
	Accuracy: 0.5625
	Network: 8.0
	Significant parcel: 182
	Accuracy: 0.567708333333
	Network: 2.0
	Significant parcel: 192
	Accuracy: 0.5625
	Network: 6.0
	Significant parcel: 214
	Accuracy: 0.5625
	Network: 9.0
	Significant parcel: 217
	Accuracy: 0.567708333333
	Network: 4.0
	Significant parcel: 237
	Accuracy: 0.583333333333
	Network: 4.0
	Significant parcel: 252
	Accuracy: 0.5625
	Network: 9.0
	Significant parcel: 270
	Accuracy: 0.567708333333
	Network: 9.0
	Significant parcel: 276
	Accuracy: 0.567708333333
	Network: 5.0
	Significant parcel: 335
	Accuracy: 0.567708333333
	Network: 9.0
	Significant parcel: 356
	Accuracy: 0.567708333333
	Network: 9.0
Network mappings: {'vis2': 2, 'lan': 

## 1.1 Run across subject decoding on left-hand motor responses

In [ ]:
nproc = 20
ncvs = 1

rois = np.where(networkdef==networkmappings['smn'])[0] + 1
rois = [189] # Right S1


distances_baseline_lh = []
for roi in range(nParcels):
    print 'Decoding roi', roi
    if roi in bad_rois: 
        distances_baseline_lh.append(np.zeros((1,len(subjNums)*2.0)))
        continue

    distances_baseline_lh.append(motorResponseDecodings(stats.zscore(data_task_rh,axis=0), 
                                                     stats.zscore(actflow_data_rh[:,:,:,roi],axis=0),
                                                     rois=rois, ncvs=ncvs, nproc=nproc))
    
distances_baseline_lh = np.squeeze(np.asarray(distances_baseline_lh))

Decoding roi 0
Decoding roi 1
Decoding roi 2
Decoding roi 3


## 1.2 Compute statistics

In [32]:
statistics_rh = np.zeros((distances_baseline.shape[0],3))
for roicount in range(distances_baseline.shape[0]):
    ntrials = len(subjNums)*2
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=0.5)
    if np.mean(distances_baseline[roicount,:])>0.5:
        p = p/2.0
    else:
        p = 1.0-p/2.0

    statistics_rh[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics_rh[roicount,1] = p

rois_notSMN = np.where(networkdef!=networkmappings['smn'])[0]
h0, qs = mc.fdrcorrection0(statistics_rh[rois_notSMN,1])
statistics_rh[:,1] = 1.0
i = 0
for roi in rois_notSMN:
    statistics_rh[roi,1] = qs[i]
    statistics_rh[roi,2] = h0[i]*statistics_rh[roi,0]
    i += 1
        
nSignificant = np.sum(statistics_rh[:,1] < 0.05)
print 'Number of significant transfers:', nSignificant

if nSignificant>0:
    sig_ind = np.where(statistics_rh[:,1]<0.05)[0]
    for ind in sig_ind:
        print '\tSignificant parcel:', ind+1
        print '\tAccuracy:', statistics_rh[ind,0]
        print '\tNetwork:', networkdef[ind]
print 'Network mappings:', networkmappings

Number of significant transfers: 0
Network mappings: {'vis2': 2, 'lan': 6, 'vis1': 1, 'none2': 12, 'none1': 11, 'dan': 5, 'aud': 8, 'pmulti': 10, 'fpn': 7, 'dmn': 9, 'smn': 3, 'con': 4}
